In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyprojroot import here

from knapsack import KnapsackGA
from knapsack.items import items

# Knapsack - Genetic Algorithm

In [ ]:
ga = KnapsackGA(items, max_weight=15, population_size=50)
print(ga)

In [ ]:
solution, result = ga.run(generations=50)
print(f"{solution=}\n{result.best_fitness=}\n{result.runtime=}s")

These are the items that made the cut:

In [ ]:
print("\n".join([i.name for i in solution]))

## Visualisations

In [ ]:
data = pd.DataFrame(result.history, columns=["generation", "best_fitness", "avg_fitness"])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sns.lineplot(data, x="generation", y="best_fitness", ax=ax, label="Best fitness")
sns.lineplot(data, x="generation", y="avg_fitness", ax=ax, label="Average fitness")

ax.set(title="Results", xlabel="Generation", ylabel="Fitness")
ax.legend(loc="upper right", bbox_to_anchor=(1.35, 1))

params = ga.params()
param_text = "Parameters\n\n" + "\n".join(f"{k}: {v}" for k, v in params.items())
ax.text(1.05, 0.8, param_text, transform=ax.transAxes, fontsize=10, verticalalignment='top')
fig.tight_layout()

In [ ]:
# Create figures/ directory if it doesn't exist
figures_dir = here("figures")
figures_dir.mkdir(parents=True, exist_ok=True)

# Create timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = figures_dir / f"ga_results_{timestamp}.png"

# Save the figure
fig.savefig(filename, bbox_inches="tight")